In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
#from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import DenseNet121

# Define the base model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout layer for regularization
predictions = Dense(5, activation='softmax')(x)  # Update the number of units to 3

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Preprocess videos and generate training data
video_dir = 'Datasett/Video'
categories = ['Programming', 'Sejarah', 'Matematika', 'Seni', 'Sains']
frames_per_video = 10  # Number of frames to extract per video

def video_generator(video_dir, categories, frames_per_video, batch_size=45):
    while True:
        X_train = []
        y_train = []
        
        for category_idx, category in enumerate(categories):
            category_path = os.path.join(video_dir, category)
            video_files = os.listdir(category_path)

            for video_file in video_files:
                video_path = os.path.join(category_path, video_file)
                cap = cv2.VideoCapture(video_path)

                # Extract frames from video
                frame_count = 0
                frames = []
                while cap.isOpened() and frame_count < frames_per_video:
                    ret, frame = cap.read()
                    if ret:
                        frame = cv2.resize(frame, (128, 128))
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        frames.append(frame)
                        frame_count += 1
                    else:
                        break

                cap.release()

                if len(frames) == frames_per_video:
                    X_train.extend(frames)
                    y_train.extend([category_idx] * frames_per_video)

        X_train = np.array(X_train)
        y_train = np.array(y_train)

        # Convert labels to one-hot encoding
        y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(categories))

        # Normalize input data
        X_train = X_train / 255.0

        num_samples = X_train.shape[0]
        indices = np.arange(num_samples)
        np.random.shuffle(indices)

        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            yield X_train[batch_indices], y_train[batch_indices]

# Preprocess validation data
valid_data = []
valid_labels = []

for category_idx, category in enumerate(categories):
    category_path = os.path.join(video_dir, category)
    video_files = os.listdir(category_path)

    for video_file in video_files:
        video_path = os.path.join(category_path, video_file)
        cap = cv2.VideoCapture(video_path)

        frame_count = 0
        frames = []
        while cap.isOpened() and frame_count < frames_per_video:
            ret, frame = cap.read()
            if ret:
                frame = cv2.resize(frame, (128, 128))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)
                frame_count += 1
            else:
                break

        cap.release()

        if len(frames) == frames_per_video:
            valid_data.extend(frames)
            valid_labels.extend([category_idx] * frames_per_video)

valid_data = np.array(valid_data)
valid_labels = np.array(valid_labels)

# Convert labels to one-hot encoding
valid_labels = tf.keras.utils.to_categorical(valid_labels, num_classes=len(categories))

# Normalize validation data
valid_data = valid_data / 255.0

# Set up early stopping
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

# Train the model
train_generator = video_generator(video_dir, categories, frames_per_video, batch_size=16)
num_train_samples = sum(len(files) for _, _, files in os.walk(video_dir))
steps_per_epoch = num_train_samples // 16
validation_data = (valid_data, valid_labels)

model.fit(train_generator, steps_per_epoch=steps_per_epoch, validation_data=validation_data, epochs=10, callbacks=[early_stopping])

# Save the model
model.save('video_classification_model.h5')


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model('video_classification_model.h5')

# Define the function to preprocess a single frame
def preprocess_frame(frame):
    frame = cv2.resize(frame, (128, 128))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = frame / 255.0
    return frame

# Define the function to preprocess a video for testing
def preprocess_video(video_path, frames_per_video):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    while cap.isOpened() and frame_count < frames_per_video:
        ret, frame = cap.read()
        if ret:
            preprocessed_frame = preprocess_frame(frame)
            frames.append(preprocessed_frame)
            frame_count += 1
        else:
            break
    cap.release()
    if len(frames) == frames_per_video:
        return np.array(frames)
    else:
        return None

# Define the video path and number of frames per video
video_path = 'Download.mp4'
frames_per_video = 10

# Preprocess the video frames
preprocessed_video = preprocess_video(video_path, frames_per_video)

if preprocessed_video is not None:
    # Make predictions on each frame individually
    predictions = []
    for frame in preprocessed_video:
        frame = np.reshape(frame, (1, 128, 128, 3))
        frame_prediction = model.predict(frame)
        predictions.append(frame_prediction)
        

    # Average the predictions for all frames
    predictions = np.array(predictions)
    average_prediction = np.mean(predictions, axis=0)

    # Get the predicted class index
    predicted_class_index = np.argmax(average_prediction)

    # Create a mapping of class indices to labels
    class_labels = ['Python', 'Sejarah', 'Matematika']

    # Get the predicted class label
    predicted_class_label = class_labels[predicted_class_index]

    # Print the predicted class label
    print("Predicted Class:", predicted_class_label)
else:
    print("Unable to preprocess the video frames.")


In [3]:
import tensorflow as tf

# Path to your Keras model file (.h5)
model_path = 'DatasetYoutubes/speech_recognition.h5'

# Output path for the TFLite model
tflite_model_path = 'DatasetYoutubes/speech_recognition.tflite'

# Load the Keras model
model = tf.keras.models.load_model(model_path)

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Include default supported ops
    tf.lite.OpsSet.SELECT_TF_OPS  # Include TensorFlow Select ops
]
tflite_model = converter.convert()

# Save the TFLite model to disk
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print('TFLite model conversion complete.')


INFO:tensorflow:Assets written to: C:\Users\patri\AppData\Local\Temp\tmp6x1_sl4t\assets


INFO:tensorflow:Assets written to: C:\Users\patri\AppData\Local\Temp\tmp6x1_sl4t\assets


TFLite model conversion complete.
